# Trigger ArcticDEM Download 

* run with arcticdem environment: aicore has geopandas/pyproj issues

In [ ]:
import ee
import os
import geopandas as gpd
import glob
%config Completer.use_jedi = False
from modules.utils import *
import datetime

In [ ]:
ee.Initialize()

### Settings

#### Paths

In [ ]:
FILEPATH = r'vector/RapidEye_Tiles.shp'
TS_FIELD = 'dem_order'

GDrive_dir_elevation = 'AI-CORE/elevation'
GDrive_dir_slope = 'AI-CORE/slope'

#### Setup GEE Datasets 

In [ ]:
slope = get_ArcticDEM_slope()

In [ ]:
rel_el_100 = get_ArcticDEM_rel_el(kernel_size=100)

#### Setup data 

In [ ]:
gdf = gpd.read_file(FILEPATH)#.to_crs('EPSG:4326')

#### filter to only one tile 

**open QGIS, select tiles and run the follwing code in the python console:**

layer = qgis.utils.iface.activeLayer()

selected_features = layer.selectedFeatures()

[f['tile_id'] for f in selected_features]

In [ ]:
field = 'tile_id'
# insert list of tile_ids to process here
idxs = []

#### run process

In [ ]:
tasks_slope = []
tasks_el = []

for idx in idxs:
    # need to drop column: 'downloaded which may contain NaN values'
    gdf_filt = gdf.query(f'{field} == {idx}')#.drop(columns='downloaded')
    tasks_slope.append(download_ee2(gdf_filt, slope, GDrive_dir=GDrive_dir_elevation, scale=2,  suffix='_slope', field=field))
    tasks_el.append(download_ee2(gdf_filt, rel_el_100, GDrive_dir=GDrive_dir_slope, scale=2,  suffix='_relative_elevation_100', field=field))
    
    gdf.loc[gdf_filt.index, TS_FIELD] = datetime.datetime.now().isoformat()

In [ ]:
print('Running Tasks:')
for tasks in [tasks_el, tasks_slope]:
    print([t.status()['description'] for t in tasks if t.status()['state'] == 'RUNNING'])

In [ ]:
print('Completed Tasks:')
for tasks in [tasks_el, tasks_slope]:
    print([t.status()['description'] for t in tasks if t.status()['state'] == 'COMPLETED'])
#[t.status()['description'] for t in tasks if t.status()['state'] == 'COMPLETED']

#### Save updated gdf 

In [ ]:
gdf.to_file(FILEPATH)

#### control

In [ ]:
gdf.dropna(subset=['dem_order'], axis=0)